In [1]:
import seaborn as sns
sns.set_palette("husl")
import matplotlib.pyplot as plt
%matplotlib inline
import hopsworks
import pandas as pd
import getpass


c:\Users\saadg\Desktop\kth-\classes\smldl\Scalable-ML\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [26]:
api_key = getpass.getpass(prompt="Enter your Hopsworks API key: ")

In [28]:
project = hopsworks.login(api_key_value=api_key)
fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/218261
Connected. Call `.close()` to terminate connection gracefully.


In [2]:
wine_df = pd.read_csv(
    "https://raw.githubusercontent.com/ID2223KTH/id2223kth.github.io/master/assignments/lab1/wine.csv"
)
wine_df

,type,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,white,7.0,0.270,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,white,6.3,0.300,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,white,8.1,0.280,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,white,7.2,0.230,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,white,7.2,0.230,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6492,red,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
6493,red,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,NaN,11.2,6
6494,red,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
6495,red,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


In [3]:
wine_df.quality.value_counts()

quality
6    2836
5    2138
7    1079
4     216
8     193
3      30
9       5
Name: count, dtype: int64

In [4]:
wine_df[wine_df["quality"]==3]

,type,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
251,white,8.5,0.260,0.21,16.20,0.074,41.0,197.0,0.99800,3.02,0.50,9.80,3
253,white,5.8,0.240,0.44,3.50,0.029,5.0,109.0,0.99130,3.53,0.43,11.70,3
294,white,9.1,0.590,0.38,1.60,0.066,34.0,182.0,0.99680,3.23,0.38,8.50,3
445,white,7.1,0.320,0.32,11.00,0.038,16.0,66.0,0.99370,3.24,0.40,11.50,3
740,white,6.9,0.390,0.40,4.60,0.022,5.0,19.0,0.99150,3.31,0.37,12.60,3
873,white,10.3,0.170,0.47,1.40,0.037,5.0,33.0,0.99390,2.89,0.28,9.60,3
1034,white,7.9,0.640,0.46,10.60,0.244,33.0,227.0,0.99830,2.87,0.74,9.10,3
1229,white,8.3,0.330,0.42,1.15,0.033,18.0,96.0,0.99110,3.20,0.32,12.40,3
1417,white,8.6,0.550,0.35,15.55,0.057,35.5,366.5,1.00010,3.04,0.63,11.00,3
1484,white,7.5,0.320,0.24,4.60,0.053,8.0,134.0,0.99580,3.14,0.50,9.10,3


In [5]:
num_cols = wine_df.select_dtypes(include=["float", "int"]).columns
for c in num_cols:
    wine_df[c].fillna(wine_df[c].mean(), inplace=True)

In [6]:
import numpy as np
from scipy import stats


def detect_outliers(data):
    z = np.abs(stats.zscore(data))
    return z > 3


outlier_indices = []

for col in num_cols:
    # Boolean array of outlier values
    outliers = detect_outliers(wine_df[col])

    # Append outlier indexes to master list
    outlier_indices.extend(wine_df[outliers].index)

# Drop duplicates
outlier_indices = list(set(outlier_indices))

# Remove outliers
wine_df.drop(outlier_indices, inplace=True)

In [7]:
wine_df.describe(include="all")

,type,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,5989,5989.000000,5989.000000,5989.000000,5989.000000,5989.000000,5989.000000,5989.000000,5989.000000,5989.000000,5989.000000,5989.000000,5989.000000
unique,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,white,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,4712,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,7.094660,0.326472,0.312820,5.505291,0.051618,30.728085,118.039239,0.994503,3.218214,0.519445,10.507619,5.844548
std,NaN,1.046806,0.145988,0.133311,4.649364,0.021014,16.366145,54.377845,0.002864,0.154365,0.127193,1.195030,0.848618
min,NaN,3.900000,0.080000,0.000000,0.600000,0.009000,1.000000,6.000000,0.987110,2.740000,0.220000,8.400000,4.000000
25%,NaN,6.400000,0.220000,0.250000,1.800000,0.037000,18.000000,84.000000,0.992160,3.110000,0.430000,9.500000,5.000000
50%,NaN,6.900000,0.290000,0.310000,3.200000,0.046000,29.000000,120.000000,0.994700,3.210000,0.500000,10.300000,6.000000
75%,NaN,7.600000,0.390000,0.380000,8.200000,0.060000,42.000000,156.000000,0.996800,3.320000,0.590000,11.300000,6.000000


In [8]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
wine_df["type"] = le.fit_transform(wine_df["type"])

In [9]:
corr = wine_df.corr()["quality"]
predictive_cols = corr[abs(corr) > 0.2].index.tolist()
print(predictive_cols)

['volatile acidity', 'chlorides', 'density', 'alcohol', 'quality']


In [10]:
final_df = wine_df[predictive_cols]
final_df = final_df.rename(columns={"volatile acidity": "volatile_acidity"})

In [11]:
final_df["quality"].value_counts()

quality
6    2636
5    1958
7    1027
8     184
4     184
Name: count, dtype: int64

In [36]:
wine_fg = fs.get_or_create_feature_group(
    name="wine",
    version=1,
    primary_key=["volatile_acidity", "chlorides", "density", "alcohol"],
    description="Wine Dataset",
)
wine_fg.insert(final_df)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/218261/fs/218180/fg/225657


Uploading Dataframe: 100.00% |██████████| Rows 5989/5989 | Elapsed Time: 00:05 | Remaining Time: 00:00


Launching job: wine_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/218261/jobs/named/wine_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x1f2b3e08850>, None)